In [1]:
import sys
sys.path.append(r'../models/research/object_detection')

import glob
import json
import re
import tensorflow as tf
from utils import dataset_util

In [2]:
def printRecursive(obj, depth=0):
    if isinstance(obj, dict):
        for k, v in obj.items():
            spacer = "  " * depth
            print("{}{}: {}".format(spacer, k, type(v)))
            printRecursive(v, depth + 1)
    if isinstance(obj,list):
        for v in obj:
            printRecursive(v, depth + 1)
            
            
for annotation_json_path in glob.glob("./data-manabiya/width1000/json/*.json"):
    print(annotation_json_path)
    json_dict = json.load(open(annotation_json_path, 'r'))
    printRecursive(json_dict)

./data-manabiya/width1000/json/85228d61-209f-4675-b30b-fd842b5623a1.json
input_data_id: <type 'unicode'>
comment: <type 'unicode'>
task_id: <type 'unicode'>
detail: <type 'list'>
    comment: <type 'unicode'>
    user_id: <type 'unicode'>
    account_id: <type 'unicode'>
    annotation_id: <type 'unicode'>
    label_name: <type 'unicode'>
    label_id: <type 'unicode'>
    data_holding_type: <type 'unicode'>
    data: <type 'unicode'>
    additional_data_list: <type 'list'>
        type: <type 'unicode'>
        choice: <type 'unicode'>
        additional_data_definition_name: <type 'unicode'>
        choice_name: <type 'unicode'>
        additional_data_definition_id: <type 'unicode'>
    comment: <type 'unicode'>
    user_id: <type 'unicode'>
    account_id: <type 'unicode'>
    annotation_id: <type 'unicode'>
    label_name: <type 'unicode'>
    label_id: <type 'unicode'>
    data_holding_type: <type 'unicode'>
    data: <type 'unicode'>
    additional_data_list: <type 'list'>
     

        choice_name: <type 'unicode'>
        additional_data_definition_id: <type 'unicode'>
    comment: <type 'unicode'>
    user_id: <type 'unicode'>
    account_id: <type 'unicode'>
    annotation_id: <type 'unicode'>
    label_name: <type 'unicode'>
    label_id: <type 'unicode'>
    data_holding_type: <type 'unicode'>
    data: <type 'unicode'>
    additional_data_list: <type 'list'>
        type: <type 'unicode'>
        choice: <type 'unicode'>
        additional_data_definition_name: <type 'unicode'>
        choice_name: <type 'unicode'>
        additional_data_definition_id: <type 'unicode'>
    comment: <type 'unicode'>
    user_id: <type 'unicode'>
    account_id: <type 'unicode'>
    annotation_id: <type 'unicode'>
    label_name: <type 'unicode'>
    label_id: <type 'unicode'>
    data_holding_type: <type 'unicode'>
    data: <type 'unicode'>
    additional_data_list: <type 'list'>
        type: <type 'unicode'>
        choice: <type 'unicode'>
        additional_data_de

AFのアノテーション、現在はこんな構造。これを SSD 用に変換する

```
input_data_id: <type 'unicode'>
comment: <type 'unicode'>
task_id: <type 'unicode'>
detail: <type 'list'>
        comment: <type 'unicode'>
        user_id: <type 'unicode'>
        account_id: <type 'unicode'>
        annotation_id: <type 'unicode'>
        label_name: <type 'unicode'>
        label_id: <type 'unicode'>
        data_holding_type: <type 'unicode'>
        data: <type 'unicode'>
        additional_data_list: <type 'list'>
                type: <type 'unicode'>
                choice: <type 'unicode'>
                additional_data_definition_name: <type 'unicode'>
                choice_name: <type 'unicode'>
                additional_data_definition_id: <type 'unicode'>
```

これによれば、TF Object Detectionに自分のデータセットを学習させるためには、データTFRecords という形式にする必要がある。
https://www.tensorflow.org/api_guides/python/python_io#tfrecords_format_details

SSDのデータセット構造

In [3]:
re_example_type = re.compile("^(\S+)\s")
re_example_value = re.compile("^\S+\s\{\n\s+value:(.*)\n\}")

def truncate(s, limit):
    if len(s) > limit:
        return s[:limit] + "... (truncated)"
    else:
        return s

for record in tf.python_io.tf_record_iterator("../models/research/object_detection/test_data/pets_examples.record"):
    example = tf.train.Example.FromString(record)
    
    print("見本:")
    for key in sorted(example.features.feature):
        v = example.features.feature[key]
        vtype = re_example_type.search(str(v)).group(0)
        vvalue = re_example_value.search(str(v)).group(1)
        print("  {}: {} = {}".format(key, vtype, truncate(vvalue, 50)))

    print("raw:")
    for key in sorted(example.features.feature):
        v = example.features.feature[key]
        print("  {}: {}".format(key, v))

    # 型と見本を見たいだけなので1個で終了    
    break    

見本:
  image/encoded: bytes_list  =  "\377\330\377\340\000\020JFIF\000\001\001\000\000... (truncated)
  image/filename: bytes_list  =  "chihuahua_153.jpg"
  image/format: bytes_list  =  "jpeg"
  image/height: int64_list  =  225
  image/key/sha256: bytes_list  =  "47e3bd3f62140d5fac89ed0148d2a70c55c5a7064b45b168... (truncated)
  image/object/bbox/xmax: float_list  =  0.759999990463
  image/object/bbox/xmin: float_list  =  0.396666675806
  image/object/bbox/ymax: float_list  =  0.568888902664
  image/object/bbox/ymin: float_list  =  0.239999994636
  image/object/class/label: int64_list  =  11
  image/object/class/text: bytes_list  =  "chihuahua"
  image/object/difficult: int64_list  =  0
  image/object/truncated: int64_list  =  0
  image/object/view: bytes_list  =  "Frontal"
  image/source_id: bytes_list  =  "chihuahua_153.jpg"
  image/width: int64_list  =  300
raw:
  image/encoded: bytes_list {
  value: "\377\330\377\340\000\020JFIF\000\001\001\000\000\001\000\001\000\000\377\376\000;CRE

In [4]:
def bounding_box(coordinates):
    _x = 0
    _y = 1
    _xs = [t[_x] for t in coordinates]
    _ys = [t[_y] for t in coordinates]
    min_x, min_y = min(_xs), min(_ys)
    max_x, max_y = max(_xs), max(_ys)
    return (min_x, min_y, max_x, max_y)

bounding_box([(1,1), (2,4), (3,3)])

(1, 1, 3, 4)

In [5]:
input_data_id_2_input_name = {
    "2e152501-d5d9-4026-bde0-7c28c1e773e0":"540782936.273833.png",
    "41816be2-3f57-4be9-bfd3-d228a4c1ca69":"540782823.324211.png",
    "418fc9c8-3ec4-49b2-8176-a8cf5376c026":"540782586.412328.png",
    "85228d61-209f-4675-b30b-fd842b5623a1":"540782457.810163.png",
    "b257a1e9-433d-480d-a41e-ef1dbf29d67e":"540782780.456003.png",
    "b4c190e3-9599-4d7f-9d97-d23b09752fdc":"540782910.323475.png"
}
class_text_2_index = {label: index for (index, label) in enumerate([
    "january-normal", "january-poetry-ribbon", "january-special-crane-and-sun",
    "february-normal", "february-poetry-ribbon", "february-special-bush-warbler-in-a-tree",
    "march-normal", "march-poetry-ribbon", "march-special-camp-curtain",
    "april-normal", "april-red-ribbon", "april-special-cuckoo",
    "may-normal", "may-red-ribbon", "may-special-water-irs-and-eight-plank-bridge",
    "june-normal", "june-purple-ribbon", "june-special-butterflies",
    "july-normal", "july-red-ribbon", "july-special-boar", 
    "august-normal", "august-special-geese-in-flight", "august-special-full-moon-with-red-sky",
    "september-normal", "september-purple-ribbon", "september-special-poetry-sake-cup",
    "october-normal", "october-purple-ribbon", "october-special-deer-and-maple",
    "november-red-ribbon", "november-special-lightning", "november-special-swallow", "november-special-rainman-with-umbrella-and-frog",
    "december-normal", "december-special-chinese-phoenix"
], 1)}

In [6]:
class_text_2_index

{'april-normal': 10,
 'april-red-ribbon': 11,
 'april-special-cuckoo': 12,
 'august-normal': 22,
 'august-special-full-moon-with-red-sky': 24,
 'august-special-geese-in-flight': 23,
 'december-normal': 35,
 'december-special-chinese-phoenix': 36,
 'february-normal': 4,
 'february-poetry-ribbon': 5,
 'february-special-bush-warbler-in-a-tree': 6,
 'january-normal': 1,
 'january-poetry-ribbon': 2,
 'january-special-crane-and-sun': 3,
 'july-normal': 19,
 'july-red-ribbon': 20,
 'july-special-boar': 21,
 'june-normal': 16,
 'june-purple-ribbon': 17,
 'june-special-butterflies': 18,
 'march-normal': 7,
 'march-poetry-ribbon': 8,
 'march-special-camp-curtain': 9,
 'may-normal': 13,
 'may-red-ribbon': 14,
 'may-special-water-irs-and-eight-plank-bridge': 15,
 'november-red-ribbon': 31,
 'november-special-lightning': 32,
 'november-special-rainman-with-umbrella-and-frog': 34,
 'november-special-swallow': 33,
 'october-normal': 28,
 'october-purple-ribbon': 29,
 'october-special-deer-and-maple':

In [7]:
def byte_encode_image(imagepath):
    import io 
    from PIL import Image

    with open(imagepath, "r") as imageFile:
        img = Image.open(imageFile, mode='r')    
        imgByteArr = io.BytesIO()
        img.save(imgByteArr, format='PNG')
        return imgByteArr.getvalue()

In [9]:
tfrecords_out_path = "af_dataset.record"

with tf.python_io.TFRecordWriter(tfrecords_out_path) as writer:
    for annotation_json_path in glob.glob("./data-manabiya/width1000/json/*.json"):
        json_dict = json.load(open(annotation_json_path, 'r'))
        filename = input_data_id_2_input_name[json_dict["input_data_id"]]
        image_path = "./data-manabiya/width1000/image/{}".format(filename)
        # 画像サイズは決め打ち
        width,height = 1000.0, 750.0
        # TODO: encode方法あってるか確かめたいが未調査
        encoded_image_data = byte_encode_image(image_path)
        
        # 1画像内の複数アノテーションを扱うため配列
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []
        
        for annotation in json_dict["detail"]:           
            flat_coordinates = annotation["data"].split(",")
            xy_coordinates = [(int(x),int(y)) for (x,y) in zip(*[iter(flat_coordinates)]*2)]
            min_x, min_y, max_x, max_y = bounding_box(xy_coordinates)
            xmins.append(min_x / width)
            ymins.append(min_y / height)
            xmaxs.append(max_x / width)
            ymaxs.append(max_y / height)
            class_text = str(annotation["additional_data_list"][0]["choice"])
            classes_text.append(class_text)
            classes.append(class_text_2_index[class_text])
            
        # import matplotlib.pyplot as plt
        # import numpy as np
        # from PIL import Image
        # im = Image.open(image_path, "r")

        example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(int(height)),
            'image/width': dataset_util.int64_feature(int(width)),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_image_data),
            'image/format': dataset_util.bytes_feature("png"),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        writer.write(example.SerializeToString())

# # TODO: 以下は、座標データを囲むbounding boxになっているか確認用にプロットしている。確認できたらTFRecordに書き込めばいいのでプロット不要
# %matplotlib inline
# plt.imshow(np.array(im))

### output Label-Map

In [10]:
index_2_class_text = {index: label for (label, index) in class_text_2_index.items()}
index_2_class_text

{1: 'january-normal',
 2: 'january-poetry-ribbon',
 3: 'january-special-crane-and-sun',
 4: 'february-normal',
 5: 'february-poetry-ribbon',
 6: 'february-special-bush-warbler-in-a-tree',
 7: 'march-normal',
 8: 'march-poetry-ribbon',
 9: 'march-special-camp-curtain',
 10: 'april-normal',
 11: 'april-red-ribbon',
 12: 'april-special-cuckoo',
 13: 'may-normal',
 14: 'may-red-ribbon',
 15: 'may-special-water-irs-and-eight-plank-bridge',
 16: 'june-normal',
 17: 'june-purple-ribbon',
 18: 'june-special-butterflies',
 19: 'july-normal',
 20: 'july-red-ribbon',
 21: 'july-special-boar',
 22: 'august-normal',
 23: 'august-special-geese-in-flight',
 24: 'august-special-full-moon-with-red-sky',
 25: 'september-normal',
 26: 'september-purple-ribbon',
 27: 'september-special-poetry-sake-cup',
 28: 'october-normal',
 29: 'october-purple-ribbon',
 30: 'october-special-deer-and-maple',
 31: 'november-red-ribbon',
 32: 'november-special-lightning',
 33: 'november-special-swallow',
 34: 'november-sp

In [11]:
label_map_filepath = "label_map.pbtxt"

with open(label_map_filepath, "w") as f:
    for index in sorted(index_2_class_text.keys()):
        label = index_2_class_text[index]
        f.write("item {\n")
        f.write("  id: {}\n".format(index))
        f.write("  name: '{}'\n".format(label))
        f.write("}\n")